Импорт плагинов

In [10]:
import folium
import pandas as pd
import geopandas as gpd

from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.plugins import GroupedLayerControl

from shapely import geometry

Импорт уровней, задавание их имени

In [11]:
class geojson_load:
    def __init__(self, layer_name, layer_path):
        self.name = layer_name
        self.data = gpd.read_file(layer_path)

okn_points = geojson_load(
    layer_name = 'Точки ОКН',
    layer_path ='data/points.geojson')

okn_buildings = geojson_load(
    layer_name = 'Контур ОКН-объектов капитального строительства',
    layer_path = 'data/buildings.geojson')

hotels = geojson_load(
    layer_name = 'Теплокарта концентрации гостиниц',
    layer_path = 'data/hotels.geojson')

admin_boundary = geojson_load(
    layer_name = 'Административная граница',
    layer_path = 'data/admin_boundary.geojson')

grid_layer_name = 'Число ОКН агрегированное по сетке'


Настройки карты

In [12]:
zoom_start = 14
tiles = 'https://api.mapbox.com/styles/v1/heizern/clfjq8u76000601pdnotlz682/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiaGVpemVybiIsImEiOiJja283cmh5OWcxeW8xMnhscHdlcnB2ZWp6In0.7XU4-TcOsJKNHaME3CAayg'
attr = (
    '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> '
    'contributors, &copy; <a href="https://www.mapbox.com/about/maps/">Mapbox</a>'
)
start_loc = [51.758, 55.104]
square_size = 300

m = folium.Map(location = start_loc, 
               zoom_start = zoom_start, tiles = tiles, control_scale = True, attr = attr)

In [15]:
# Проверка
m

Добавление административных границ

In [14]:
folium.GeoJson(admin_boundary.data,
               name=admin_boundary.name, 
               control=False, 
               style_function=lambda x: {
        "color": 'black', 'weight': '2', 'fillOpacity' :0
    }).add_to(m)

Создание сетки и подсчет числа памятников

In [ ]:
minX, minY, maxX, maxY = okn_points.data.total_bounds
grid_cells = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32640')
fishnet['id'] = fishnet.index
merged = gpd.sjoin(okn_points.data, fishnet, how='left', predicate='within')
merged['n'] = 1
merged = merged.dissolve(by="index_right", aggfunc="count")
fishnet.loc[merged.index, 'n'] = merged.n.values

Добавление сетки на карту

In [ ]:
grid_layer = folium.Choropleth(
    geo_data=fishnet,
    data=fishnet,
    columns=['id', 'n'],
    fill_color='YlGnBu',
    fill_opacity = 0.5,
    key_on='id',
    nan_fill_opacity=0,
   line_color = "#0000",
   legend_name="amount of heritage sites",
   name=grid_layer_name
)

Добавление теплокарты отелей на карту

In [ ]:
hotels.data = hotels.data.to_crs('EPSG:4326')
hotels.data = hotels.data.dropna()
HeatMap(zip(hotels.data.geometry.y, hotels.data.geometry.x)).add_to(m)

Добавление слоя ОКН - объектов капитального строительства

In [ ]:
folium.GeoJson(
    okn_buildings.data,
    name=okn_buildings.name,
    style_function=lambda x: {
        "fillColor": 'yellow' if x['properties']['category']
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
    show=False,
).add_to(m)

In [16]:
okn_buildings.data.columns

Index(['fid', 'full_id', 'is_ensemble', 'geometry'], dtype='object')

In [ ]:
# Поскольку кластеры с tooltip и popup нельзя создать напрямую из json происходят дополнительные действия

# Создание листа с popup
temp_list = []
for column, alias in zip(['name','registry_number','address','category','type'],['Название:', 'Номер в реестре:','Адрес:', 'Уровень значения:', 'Тип ОКН:']):
    if column != 'type':
        temp_list.append(okn_points.data[column].apply(lambda x: f"""<b>{alias}</b>: {x} <br>"""))
    else:
        temp_list.append(okn_points.data[column].apply(lambda x: f"""<b>{alias}</b>: {x}"""))
popup_list = pd.concat(temp_list,axis=1).apply(lambda x: ''.join(x), axis=1).to_list()

# Создание листа с tooltip
tooltip_list = okn_points.data['name'].apply(lambda x: f""""<b>{x}<b>""").to_list()

# Создание MarkerCluster
okn_points.data = okn_points.data.to_crs("EPSG:4326")
okn_points_cluster = MarkerCluster(
    name=okn_points.name,
    locations = zip(okn_points.data.geometry.y, okn_points.data.geometry.x),
    tooltips = tooltip_list,
    popups = popup_list
)
m.add_child(okn_points_cluster)



Добавление виджетов

In [ ]:
from folium.plugins import MousePosition
from folium.plugins import Fullscreen

MousePosition().add_to(m)
Fullscreen(
    position="bottomright",
    title="тык",
    title_cancel="тудык",
    force_separate_button=True,
).add_to(m)

In [9]:
m

In [ ]:
okn_points.data[['name','registry_number','address','category','type']]

In [ ]:
folium.GeoJson(
    okn_points.data,
    marker = folium.Circle(radius=3, fill=True, weight= 0, fillOpacity =1 ),
    popup = folium.GeoJsonPopup(fields=['name','registry_number','address','category','type'],
                                aliases=['Название:', 'Номер в реестре:','Адрес:', 'Уровень значения:', 'Тип ОКН:']
                                ),
    tooltip = folium.GeoJsonTooltip(fields=['name'],aliases=['Название:']),
).add_to(m)

In [ ]:
okn_points.data['type'].unique()

Создание карты

In [7]:
m